## Baseline prediction

In [ ]:
import transformers
from pathlib import Path
#from transformers import pipeline
import time
import torch
print(transformers.__version__)

In [ ]:
base_directory = Path.cwd().parent

dataset_name = "yale_econ251"
data_dir = base_directory / 'data'

dataset_size = "normal" # or 'tiny'

if dataset_size == "tiny":
    audio_dir = data_dir / 'inputs' / dataset_name / 'lectures-tiny'
    transcripts_dir = data_dir / 'inputs' / dataset_name / 'transcripts-tiny'
else:
    audio_dir = data_dir / 'inputs' / dataset_name / 'lectures'
    transcripts_dir = data_dir / 'inputs' / dataset_name / 'transcripts'

In [ ]:
# load the model pipeline
# model_name = "facebook/wav2vec2-base-960h" # baseline
# model_name = "facebook/wav2vec2-base"
#model_name = "facebook/wav2vec2-base-100h"

#model_name = "facebook/wav2vec2-large-robust-ft-libri-960h"
model_name = "facebook/wav2vec2-large-960h"

pipe = transformers.pipeline("automatic-speech-recognition", model=model_name,device =0)

In [ ]:
transcripts_dir

In [ ]:
pred_folder_name = model_name.split('/')[1]
print(pred_folder_name)
predictions_dir = data_dir / 'predictions' / dataset_name /  pred_folder_name / dataset_size
print(predictions_dir)
if not predictions_dir.is_dir():
    predictions_dir.mkdir(parents=True)

In [ ]:
chunk_length = 10

#selected_files = ['17','18','19','20','21','22','23','24','25','26']#, '07', '08', '09', '10', '11', '12', '13','14','15',
                  #'16','17','18','19','20','21','22','23','24','25','26' ]

for audio_file in audio_dir.glob('*.mp3'):
    # print(audio_file)
    file_no = audio_file.stem.split('_')[1]
   # if file_no not in selected_files:
    #    continue

    print(f"Extracting text from speech for {audio_file}")
    start = time.time()
    output = pipe(str(audio_file), chunk_length_s=chunk_length)
    text = output['text']
    time_taken = time.time()-start
    print(f"Speech recognition took {time.time()-start} seconds")

    print('Saving text to file')
    file_name = 'pred_' + "_" + file_no +"_" + dataset_size #+ '_cl10'
    out_file = (predictions_dir / file_name).with_suffix('.txt')
    print(out_file)
    if not out_file.is_file():
        print(f"Saved at {out_file}")
        with open(out_file, 'w') as f:
            f.write(text)
    else:
        print(f"{out_file} already exists")